In [18]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time

def process_bar(percent, start_str='', end_str='', total_length=0):
    bar = ''.join(
        ["\033[31m%s\033[0m" % '   '] * int(percent * total_length)) + ''
    bar = '\r' + start_str + bar.ljust(total_length) + ' {:0>4.1f}%|'.format(
        percent * 100) + end_str
    print(bar, end='', flush=True)

headers = {
    'User-Agent':
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
                           (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'
}

data = []
attrs = ["户型", "面积", "楼层", "朝向", "建成时间", "经纪人", "地址", "单价"]

for num in range(1,39):
    meanUrl = "https://xj.esf.fang.com/house/i3%s"%num
    meanPage = requests.get(meanUrl, headers=headers)
    print("页面状态码:{0}".format(meanPage.status_code))
    soup = BeautifulSoup(meanPage.text, "html.parser")
    tag = soup.find_all("script")[3]
    a = str(tag).find("rfss")
    var_t4 = meanUrl
    var_t3 = str(tag)[a:a + 28]
    newUrl = var_t4 + "?" + var_t3
    print("当前访问为 {0}:".format(newUrl))
    newPage = requests.get(newUrl, headers=headers)
    newSoup = BeautifulSoup(newPage.text, "html.parser")
    fangData = []
    n =60
    for i in range(n):
        try:
            p = newSoup.find_all("dl", {"class": "clearfix"})[i].dd.p  #首页信息
            span = newSoup.find_all("p", {"class": "add_shop"})[i]  #地址信息
            info = p.text.replace("\t", "").replace("\n",
                                                    "").split("|")  #信息汇总(list)
            price = newSoup.find_all("dd", {"class": "price_right"})[i]  #单价
            info.append(span.span.text)
            info.append(price.span.next_sibling.next_sibling.text)
            data.append(info)
        except:
            pass
        end_str = '100%'
        process_bar(i/(n-1), start_str='', end_str=end_str, total_length=15)
    time.sleep(2)
    
df = pd.DataFrame(data=data,columns=attrs)
df

页面状态码:200
当前访问为 https://xj.esf.fang.com/house/i31?rfss=1-b082af0a9eb8b4af6f-5e:
                                              100.0%|100%页面状态码:200
当前访问为 https://xj.esf.fang.com/house/i32?rfss=1-080cad89993f352e1d-5d:
                                              100.0%|100%页面状态码:200
当前访问为 https://xj.esf.fang.com/house/i33?rfss=1-340e280c9c3bb6ad6c-5c:
                                              100.0%|100%页面状态码:200
当前访问为 https://xj.esf.fang.com/house/i34?rfss=1-8392ab889bb8332d08-5b:
                                              100.0%|100%页面状态码:200
当前访问为 https://xj.esf.fang.com/house/i35?rfss=1-aa15ab8c9d3a342e59-5a:
                                              100.0%|100%页面状态码:200
当前访问为 https://xj.esf.fang.com/house/i36?rfss=1-83192c88983e36a80b-59:
                                              100.0%|100%页面状态码:200
当前访问为 https://xj.esf.fang.com/house/i37?rfss=1-a51a29089a39b42b4a-58:
                                              100.0%|100%页面状态码:200
当前访问为 https://xj.esf.fang.com/h

户型       面积        楼层   朝向    建成时间  经纪人                     地址  \
0    2室2厅   100.3㎡   高层（共5层）  南北向  2015年建   王滨        南湖-水磨沟区高尔夫路638号   
1    6室3厅     233㎡  低层（共18层）  南北向  2016年建   王滨           喀什东路-新市区喀什东路   
2    3室2厅     130㎡  低层（共14层）  南北向  2015年建   王滨          开发区二期-惠山巷266号   
3    2室1厅      57㎡  中层（共23层）   南向  2017年建  赵春霞           天津路-迎宾东路188号   
4    3室2厅  212.65㎡   低层（共3层）  南北向  2017年建  赵春霞  国际展览中心-红光山生态园南侧百合公寓会所   
..    ...      ...       ...  ...     ...  ...                    ...   
542  3室2厅  111.25㎡  高层（共33层）  南北向  2023年建   李娟                银川路-泉州街   
543  1室1厅   79.89㎡  高层（共31层）  西南向  2023年建   苏哲                银川路-泉州街   
544  3室2厅      90㎡  高层（共34层）  南北向  2016年建  蒋龙伟           喀什东路-万科·四季花城   
545  3室2厅  105.76㎡  中层（共18层）  南北向  2013年建  蒋龙伟    卡子湾-盈科梧桐山畔喀什东路车管所附近   
546  4室2厅  129.16㎡  低层（共33层）  南北向  2023年建   苏哲                银川路-泉州街   

           单价  
0    10568元/㎡  
1     8583元/㎡  
2    16923元/㎡  
3     8684元/㎡  
4    21161元/㎡  
..        ...  
542   8719元/㎡  
543   7510元/㎡  
544   8333元/㎡  
545   7753元/㎡  
546   8903元/㎡  

[547 rows x 8 columns]

In [ ]:
#插入数据库
from sqlalchemy import create_engine
 
#填写链接信息
engine = create_engine("mysql+pymysql://root:@127.0.0.1:3306/45db?charset=utf8") 

#将数据存入数据库
df.to_sql(name = 'info',con = engine,if_exists = 'replace',index = False,index_label = False)

## 数据预处理

In [16]:
import pymysql
import pandas as pd

conn = pymysql.connect(
        host='127.0.0.1',
        user='root',
        password='',
        db='45db',
        charset='utf8'
    )
df = pd.read_sql("select * from info",conn)
df

户型       面积        楼层   朝向    建成时间  经纪人                     地址  \
0    2室2厅   100.3㎡   高层（共5层）  南北向  2015年建   王滨        南湖-水磨沟区高尔夫路638号   
1    6室3厅     233㎡  低层（共18层）  南北向  2016年建   王滨           喀什东路-新市区喀什东路   
2    3室2厅     130㎡  低层（共14层）  南北向  2015年建   王滨          开发区二期-惠山巷266号   
3    2室1厅      57㎡  中层（共23层）   南向  2017年建  赵春霞           天津路-迎宾东路188号   
4    3室2厅  212.65㎡   低层（共3层）  南北向  2017年建  赵春霞  国际展览中心-红光山生态园南侧百合公寓会所   
..    ...      ...       ...  ...     ...  ...                    ...   
542  3室2厅  111.25㎡  高层（共33层）  南北向  2023年建   李娟                银川路-泉州街   
543  1室1厅   79.89㎡  高层（共31层）  西南向  2023年建   苏哲                银川路-泉州街   
544  3室2厅      90㎡  高层（共34层）  南北向  2016年建  蒋龙伟           喀什东路-万科·四季花城   
545  3室2厅  105.76㎡  中层（共18层）  南北向  2013年建  蒋龙伟    卡子湾-盈科梧桐山畔喀什东路车管所附近   
546  4室2厅  129.16㎡  低层（共33层）  南北向  2023年建   苏哲                银川路-泉州街   

           单价  
0    10568元/㎡  
1     8583元/㎡  
2    16923元/㎡  
3     8684元/㎡  
4    21161元/㎡  
..        ...  
542   8719元/㎡  
543   7510元/㎡  
544   8333元/㎡  
545   7753元/㎡  
546   8903元/㎡  

[547 rows x 8 columns]

In [17]:
def huxing2shi(x):
    return x[0]

def huxing2ting(x):
    return x[2]

def mianji2num(x):
    return x.strip('㎡')

def louceng(x):
    return x.split('（')[0]

def louceng2num(x):
    return x.split('（')[1].strip('共').strip('层）')

def year(x):
    return x.strip('年建')
    
def price(x):
    return x.strip('元/㎡')

df['室'] = df['户型'].apply(huxing2shi)
df['厅'] = df['户型'].apply(huxing2ting)
df['面积'] = df['面积'].apply(mianji2num)
df['总楼层'] = df['楼层'].apply(louceng2num)
df['楼层'] = df['楼层'].apply(louceng)
df['建成时间'] = df['建成时间'].apply(year)
df['单价'] = df['单价'].apply(price)
df

户型      面积  楼层   朝向  建成时间  经纪人                     地址     单价  室  厅 总楼层
0    2室2厅   100.3  高层  南北向  2015   王滨        南湖-水磨沟区高尔夫路638号  10568  2  2   5
1    6室3厅     233  低层  南北向  2016   王滨           喀什东路-新市区喀什东路   8583  6  3  18
2    3室2厅     130  低层  南北向  2015   王滨          开发区二期-惠山巷266号  16923  3  2  14
3    2室1厅      57  中层   南向  2017  赵春霞           天津路-迎宾东路188号   8684  2  1  23
4    3室2厅  212.65  低层  南北向  2017  赵春霞  国际展览中心-红光山生态园南侧百合公寓会所  21161  3  2   3
..    ...     ...  ..  ...   ...  ...                    ...    ... .. ..  ..
542  3室2厅  111.25  高层  南北向  2023   李娟                银川路-泉州街   8719  3  2  33
543  1室1厅   79.89  高层  西南向  2023   苏哲                银川路-泉州街   7510  1  1  31
544  3室2厅      90  高层  南北向  2016  蒋龙伟           喀什东路-万科·四季花城   8333  3  2  34
545  3室2厅  105.76  中层  南北向  2013  蒋龙伟    卡子湾-盈科梧桐山畔喀什东路车管所附近   7753  3  2  18
546  4室2厅  129.16  低层  南北向  2023   苏哲                银川路-泉州街   8903  4  2  33

[547 rows x 11 columns]

In [18]:
#插入数据库
from sqlalchemy import create_engine
 
#填写链接信息
engine = create_engine("mysql+pymysql://root:@127.0.0.1:3306/45db?charset=utf8") 

#将数据存入数据库
df.to_sql(name = 'clear_info',con = engine,if_exists = 'replace',index = False,index_label = False)